# 10_Sentiment_Analysis

# Workd on it on 23 Aug 2021

In [2]:
import pandas as pd
df=pd.read_csv("Tweets.csv")

In [3]:
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [4]:
df1=df[["airline_sentiment","text"]]

In [6]:
df1.shape

(14640, 2)

In [7]:
df1["airline_sentiment"].value_counts()

negative    9178
neutral     3099
positive    2363
Name: airline_sentiment, dtype: int64

In [8]:
dict={"negative":0, "neutral":1,"positive":2}

In [10]:
df1["airline_sentiment"]=df1["airline_sentiment"].map(dict)

<ipython-input-10-86e4df9e0527>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["airline_sentiment"]=df1["airline_sentiment"].map(dict)


In [11]:
df1["airline_sentiment"].value_counts()

0    9178
1    3099
2    2363
Name: airline_sentiment, dtype: int64

In [13]:
#Cleaning the text
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
#from nltk.stem import WordNetLemmatizer

ps=PorterStemmer()
corpus=[]
for i in range(len(df1)):
    review=re.sub('[^a-zA-Z]',' ',df1["text"][i])
    review=review.lower()
    review=review.split()
    review=[ps.stem(word) for word in review if word not in set(stopwords.words("English"))]
    review=' '.join(review)
    corpus.append(review)

In [14]:
corpus[1]

'virginamerica plu ad commerci experi tacki'

# basic ML model

In [15]:
# Creating bag of words
from sklearn.feature_extraction.text import CountVectorizer
cv= CountVectorizer(max_features=2500) # out of 6296, we will take top 2500 frequent
X=cv.fit_transform(corpus).toarray()

In [16]:
# Train test split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train,y_test=train_test_split(X, df1["airline_sentiment"], test_size=0.20, random_state=0)

In [17]:
# training model using Naive bayes classifier

from sklearn.naive_bayes import MultinomialNB
spm_model=MultinomialNB().fit(X_train,y_train)

y_pred=spm_model.predict(X_test)

from sklearn.metrics import confusion_matrix

conf=confusion_matrix(y_test,y_pred)

from sklearn.metrics import accuracy_score

acc=accuracy_score(y_test,y_pred)

In [18]:
print(conf)
print(acc)

[[1645  161   64]
 [ 232  325   57]
 [  94   55  295]]
0.7735655737704918


# Hybrid model

In [20]:
import re
from collections import Counter
Counter(" ".join(corpus).split()).most_common(100)

[('flight', 4880),
 ('unit', 4168),
 ('usairway', 3054),
 ('americanair', 2964),
 ('southwestair', 2461),
 ('jetblu', 2397),
 ('thank', 1703),
 ('get', 1623),
 ('co', 1216),
 ('http', 1211),
 ('hour', 1170),
 ('cancel', 1065),
 ('help', 1063),
 ('delay', 1012),
 ('servic', 1004),
 ('time', 988),
 ('custom', 948),
 ('call', 792),
 ('bag', 771),
 ('wait', 754),
 ('plane', 735),
 ('fli', 714),
 ('us', 705),
 ('need', 689),
 ('hold', 687),
 ('amp', 683),
 ('go', 639),
 ('one', 589),
 ('still', 586),
 ('tri', 586),
 ('day', 577),
 ('pleas', 576),
 ('would', 561),
 ('airlin', 559),
 ('gate', 543),
 ('back', 534),
 ('flightl', 526),
 ('virginamerica', 524),
 ('seat', 519),
 ('book', 515),
 ('make', 483),
 ('check', 473),
 ('chang', 468),
 ('got', 468),
 ('phone', 460),
 ('like', 453),
 ('hr', 435),
 ('today', 433),
 ('guy', 426),
 ('late', 426),
 ('agent', 421),
 ('work', 412),
 ('miss', 411),
 ('ticket', 401),
 ('know', 396),
 ('airport', 388),
 ('min', 384),
 ('travel', 380),
 ('take', 378)

In [21]:
p1=r"thank"
p2=r"like"
p3=r"servic"
p4=r"great"
p5=r"good"
p6=r"love"

positive_pattern_list=[p1,p2,p3,p4,p5,p6]

In [22]:
positive_complex_pattern=re.compile('|'.join(['%s' % i for i in positive_pattern_list]), re.IGNORECASE)

In [23]:
positive_complex_pattern

re.compile(r'thank|like|servic|great|good|love', re.IGNORECASE|re.UNICODE)

In [24]:
n1=r"cancel"
n2=r"help"
n3=r"delay"
n4=r"wait"
n5=r"pleas"
n6=r"hold"

negative_pattern_list=[p1,p2,p3,p4,p5,p6]

In [25]:
negative_complex_pattern=re.compile('|'.join(['%s' % i for i in negative_pattern_list]), re.IGNORECASE)

In [29]:
df1["negative_flag"]=df1["text"].apply(lambda x:1 if(len(re.findall(negative_complex_pattern,x))>0) else 0)

<ipython-input-29-2e3ab866507f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["negative_flag"]=df1["text"].apply(lambda x:1 if(len(re.findall(negative_complex_pattern,x))>0) else 0)


In [30]:
df1["positive_flag"]=df1["text"].apply(lambda x:1 if(len(re.findall(positive_complex_pattern,x))>0) else 0)

<ipython-input-30-12554c1cc067>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["positive_flag"]=df1["text"].apply(lambda x:1 if(len(re.findall(positive_complex_pattern,x))>0) else 0)


In [32]:
df1.head(100)

,airline_sentiment,text,negative_flag,positive_flag
0,1,@VirginAmerica What @dhepburn said.,0,0
1,2,@VirginAmerica plus you've added commercials t...,0,0
2,1,@VirginAmerica I didn't today... Must mean I n...,0,0
3,0,@VirginAmerica it's really aggressive to blast...,0,0
4,0,@VirginAmerica and it's a really big bad thing...,0,0
...,...,...,...,...
95,0,"@VirginAmerica Is it me, or is your website do...",1,1
96,0,@VirginAmerica I can't check in or add a bag. ...,0,0
97,0,@VirginAmerica - Let 2 scanned in passengers l...,0,0
98,0,@virginamerica What is your phone number. I ca...,0,0


In [44]:
s = pd.Series(corpus)

In [46]:
df2=df1.merge(s.rename('new'), left_index=True, right_index=True)

In [47]:
df2.head()

,airline_sentiment,text,negative_flag,positive_flag,new
0,1,@VirginAmerica What @dhepburn said.,0,0,virginamerica dhepburn said
1,2,@VirginAmerica plus you've added commercials t...,0,0,virginamerica plu ad commerci experi tacki
2,1,@VirginAmerica I didn't today... Must mean I n...,0,0,virginamerica today must mean need take anoth ...
3,0,@VirginAmerica it's really aggressive to blast...,0,0,virginamerica realli aggress blast obnoxi ente...
4,0,@VirginAmerica and it's a really big bad thing...,0,0,virginamerica realli big bad thing


In [54]:
# Creating bag of words
from sklearn.feature_extraction.text import CountVectorizer
cv= CountVectorizer(max_features=2500) # out of 6296, we will take top 2500 frequent
X=cv.fit_transform(corpus).toarray()

In [55]:
X=pd.DataFrame(X)

In [57]:
X

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14635,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14636,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14637,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14638,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
X2=pd.concat([X,df2["positive_flag"],df2["negative_flag"]],axis=1)

In [62]:
# Train test split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train,y_test=train_test_split(X2, df1["airline_sentiment"], test_size=0.20, random_state=0)

In [63]:
# training model using Naive bayes classifier

from sklearn.naive_bayes import MultinomialNB
spm_model=MultinomialNB().fit(X_train,y_train)

y_pred=spm_model.predict(X_test)

from sklearn.metrics import confusion_matrix

conf=confusion_matrix(y_test,y_pred)

from sklearn.metrics import accuracy_score

acc=accuracy_score(y_test,y_pred)

In [64]:
print(conf)
print(acc)

[[1624  164   82]
 [ 227  320   67]
 [  87   51  306]]
0.7684426229508197
